In [1]:
from graph import *
from models import *
from metrics import *

import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

In [8]:
from copy import deepcopy

In [91]:
# load data from the spreadsheet which defines the structure of the workflow,
# as well as the parameters for data rates, efficiency, data reduction, and classifier performance
run3_system = dataframes_from_spreadsheet("cms_system_60.xlsx")
run5_system = dataframes_from_spreadsheet("cms_system_200.xlsx")

In [3]:
run5_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Compression,Link Efficiency (J/bit),Op Efficiency (J/op),PU 200
0,Tracking,Inner Tracker,1440000,40000000,0,2.220000e-11,0,1.440
1,Tracking,Outer Tracker PS,720000,40000000,0,2.220000e-11,0,0.720
2,Tracking,Outer Tracker 2S,430000,40000000,0,2.220000e-11,0,0.430
3,Tracking,Track Finder TPG,10000,40000000,0,2.220000e-11,0,0.010
4,Timing,MIP Timing BTL,240000,40000000,0,2.220000e-11,0,0.240
5,Timing,MIP Timing ETL,440000,40000000,0,2.220000e-11,0,0.440
6,Calorimetry,ECAL Barrel,600000,40000000,0,2.220000e-11,0,0.600
7,Calorimetry,HCAL Barrel,240000,40000000,0,2.220000e-11,0,0.240
8,Calorimetry,HCAL HO,30000,40000000,0,2.220000e-11,0,0.030
9,Calorimetry,HCAL HF,60000,40000000,0,2.220000e-11,0,0.060


In [4]:
#import the data predicting wall time scaling by pileup
scaling = pd.read_excel("wall time scaling.xlsx", sheet_name="Data")
#fit a polynomial to this data for CPU and GPU runtimes
fit_poly = lambda x, k3, k2, k1: k3 * x ** 3 + k2 * x ** 2 + k1 * x
k, cv = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time"])
k_gpu, cv_gpu = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time GPU"])

In [5]:
#define a dictionary with functions defining the scaling of trigger runtimes with incoming data
funcs = {"Global": lambda x: fit_poly(x, *k), "Intermediate": lambda x: x / 2.0e6}
funcs_gpu = {"Global": lambda x: fit_poly(x, *k_gpu), "Intermediate": lambda x: x / 2.0e6}

In [92]:
baseline_r3 = construct_graph(run3_system.detectors, run3_system.triggers, run3_system.globals, funcs)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [6]:
baseline = construct_graph(run5_system.detectors, run5_system.triggers, run5_system.globals, funcs)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [52]:
baseline.nodes["Intermediate"]

{'type': 'processor',
 'reduction ratio': 53.3,
 'classifier': <classifier.L1TClassifier at 0x311a96020>,
 'data reduction': 1.0,
 'op efficiency': 0.003,
 'sample data': 260000,
 'complexity': <function __main__.<lambda>(x)>,
 'global ratio': 5330.0,
 'message size': 8425000.0,
 'ops': 4.2125,
 'input rate': 39999999,
 'error matrix': array([[0.98132, 0.5152 ],
        [0.01868, 0.4848 ]]),
 'contingency': array([[39245435,     3866],
        [  747059,     3637]]),
 'discards': array([39245435,     3866]),
 'output rate': 750696,
 'energy': 0.012637500000000001,
 'power': 505499.98736250005}

In [38]:
gpu = construct_graph(run5_system.detectors, run5_system.triggers, run5_system.globals, funcs_gpu)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [138]:
baseline_2 = deepcopy(baseline)
baseline_2.nodes["Intermediate"]["reduction ratio"] = 400
baseline_2 = update_throughput(baseline_2)


In [39]:
l1t = deepcopy(baseline)
l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
l1t = update_throughput(l1t)

In [40]:
smpx = deepcopy(baseline)
smpx.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
smpx = update_throughput(smpx)

In [41]:
gpu_smpx = deepcopy(gpu)
gpu_smpx.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
gpu_smpx = update_throughput(gpu_smpx)

In [42]:
gpu_l1t = deepcopy(gpu)
gpu_l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
gpu_l1t = update_throughput(gpu_l1t)

In [43]:
smpx_l1t = deepcopy(baseline)
smpx_l1t.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
smpx_l1t = update_throughput(smpx_l1t)

In [44]:
gpu_smpx_l1t = deepcopy(gpu)
gpu_smpx_l1t.nodes["Intermediate"]["classifier"].skill_boost = 0.40
gpu_smpx_l1t.nodes["Inner Tracker"]["sample data"] *= (1 - 0.54)
gpu_smpx_l1t = update_throughput(gpu_smpx_l1t)

In [45]:
np.sum(gpu.graph["performance"][:,1])

7502

In [94]:
def extract_results(graph):
    power = (graph.graph["op power"] + graph.graph["link power"]) / density_scale_model(2032)
    confusion = graph.graph["performance"]
    acc = precision(confusion)
    rec = recall(confusion)
    f1 = f1_score(confusion)
    prod = f1 * np.sum(get_passed(confusion)) / power


    return power, acc, rec, f1, prod

In [146]:
conditions = [baseline_r3, baseline_2, baseline]

In [157]:
pileup = np.array([60, 200, 200])[:,np.newaxis]
rejection = np.array([400, 400, 53])[:,np.newaxis]

In [158]:
pileup.shape

(3, 1)

In [159]:
cond_results = np.stack([extract_results(g) for g in conditions])

In [ ]:
df2 = pd.DataFrame(cond_results.transpose(), columns = ["Pileup", "L1T Reduction Ratio", "Power (W)", "Accuracy (%)", "Recall (%)", "F1 Score (%)", "Productivity (Relevant Samples/J)"])

In [162]:
cond_results = np.concatenate((pileup, rejection, cond_results), axis=1)

In [166]:
df2 = pd.DataFrame(cond_results, columns = ["Pileup", "L1T Reduction Ratio", "Power (W)", "Accuracy (%)", "Recall (%)", "F1 Score (%)", "Productivity (Relevant Samples/J)"])

In [167]:
df2

,Pileup,L1T Reduction Ratio,Power (W),Accuracy (%),Recall (%),F1 Score (%),Productivity (Relevant Samples/J)
0,60.0,400.0,3.223451e+05,0.258741,0.259519,0.259130,0.000805
1,200.0,400.0,6.977945e+06,0.255744,0.256513,0.256128,0.000037
2,200.0,53.0,5.161452e+07,0.449507,0.449747,0.449627,0.000065


In [ ]:
extract_results(baseline_2)[-1] * 1000

0.03674207547314531

In [ ]:
extract_results(baseline)[-1] * 1000

0.06538662603179313

In [143]:
extract_results(baseline_r3)[-1] * 1000

0.8046926231004671

In [144]:
extract_results(baseline_2)[-1] * 1000

0.03674207547314531

In [145]:
extract_results(baseline)[-1] * 1000

0.06538662603179313

In [100]:
all_graphs = [baseline_r3, baseline, gpu, l1t, smpx, gpu_l1t, smpx_l1t, gpu_smpx, gpu_smpx_l1t]

In [109]:
pileup = np.array([[60, 200, 200, 200, 200, 200, 200, 200, 200,],])
rejection = np.array([[400, 53, 53, 53, 53, 53, 53, 53, 53],])
has_gpu = [False, False, True, False, False, True, False, True, True]
has_smpx = [False, False, False, False, True, False, True, True, True]
has_l1t = [False, False, False, True, False, True, True, False, True]

In [110]:
results = np.stack([extract_results(g) for g in all_graphs])

In [111]:
results

array([[3.22345063e+05, 2.58741259e-01, 2.59519038e-01, 2.59129565e-01,
        8.04692623e-04],
       [5.16145190e+07, 4.49507061e-01, 4.49746734e-01, 4.49626866e-01,
        6.53866260e-05],
       [2.58561780e+07, 4.50972555e-01, 4.51213010e-01, 4.51092751e-01,
        1.30951380e-04],
       [5.17064788e+07, 8.38563830e-01, 8.40575846e-01, 8.39568633e-01,
        1.22103772e-04],
       [4.08467796e+07, 4.49507061e-01, 4.49746734e-01, 4.49626866e-01,
        8.26233863e-05],
       [2.58466045e+07, 8.39397574e-01, 8.39509464e-01, 8.39453515e-01,
        2.43684610e-04],
       [4.08467796e+07, 4.49507061e-01, 4.49746734e-01, 4.49626866e-01,
        8.26233863e-05],
       [2.04677135e+07, 4.50972555e-01, 4.51213010e-01, 4.51092751e-01,
        1.65426499e-04],
       [2.04601393e+07, 8.39397574e-01, 8.39509464e-01, 8.39453515e-01,
        3.07838556e-04]])

In [123]:
pileup.shape

(1, 9)

In [124]:
rejection.shape

(1, 9)

In [125]:
results = np.concatenate((pileup, rejection, np.transpose(results)), axis=0)

In [126]:
results

array([[6.00000000e+01, 2.00000000e+02, 2.00000000e+02, 2.00000000e+02,
        2.00000000e+02, 2.00000000e+02, 2.00000000e+02, 2.00000000e+02,
        2.00000000e+02],
       [4.00000000e+02, 5.30000000e+01, 5.30000000e+01, 5.30000000e+01,
        5.30000000e+01, 5.30000000e+01, 5.30000000e+01, 5.30000000e+01,
        5.30000000e+01],
       [3.22345063e+05, 5.16145190e+07, 2.58561780e+07, 5.17064788e+07,
        4.08467796e+07, 2.58466045e+07, 4.08467796e+07, 2.04677135e+07,
        2.04601393e+07],
       [2.58741259e-01, 4.49507061e-01, 4.50972555e-01, 8.38563830e-01,
        4.49507061e-01, 8.39397574e-01, 4.49507061e-01, 4.50972555e-01,
        8.39397574e-01],
       [2.59519038e-01, 4.49746734e-01, 4.51213010e-01, 8.40575846e-01,
        4.49746734e-01, 8.39509464e-01, 4.49746734e-01, 4.51213010e-01,
        8.39509464e-01],
       [2.59129565e-01, 4.49626866e-01, 4.51092751e-01, 8.39568633e-01,
        4.49626866e-01, 8.39453515e-01, 4.49626866e-01, 4.51092751e-01,
        8.3

In [128]:
df = pd.DataFrame(results.transpose(), columns = ["Pileup", "L1T Reduction Ratio", "Power (W)", "Accuracy (%)", "Recall (%)", "F1 Score (%)", "Productivity (Relevant Samples/J)"])

In [129]:
df["GPU HLT"] = has_gpu
df["L1T Tracking"] = has_l1t
df["Smart Sensors"] = has_smpx

In [130]:
df

,Pileup,L1T Reduction Ratio,Power (W),Accuracy (%),Recall (%),F1 Score (%),Productivity (Relevant Samples/J),GPU HLT,L1T Tracking,Smart Sensors
0,60.0,400.0,3.223451e+05,0.258741,0.259519,0.259130,0.000805,False,False,False
1,200.0,53.0,5.161452e+07,0.449507,0.449747,0.449627,0.000065,False,False,False
2,200.0,53.0,2.585618e+07,0.450973,0.451213,0.451093,0.000131,True,False,False
3,200.0,53.0,5.170648e+07,0.838564,0.840576,0.839569,0.000122,False,True,False
4,200.0,53.0,4.084678e+07,0.449507,0.449747,0.449627,0.000083,False,False,True
5,200.0,53.0,2.584660e+07,0.839398,0.839509,0.839454,0.000244,True,True,False
6,200.0,53.0,4.084678e+07,0.449507,0.449747,0.449627,0.000083,False,True,True
7,200.0,53.0,2.046771e+07,0.450973,0.451213,0.451093,0.000165,True,False,True
8,200.0,53.0,2.046014e+07,0.839398,0.839509,0.839454,0.000308,True,True,True


In [88]:
df["Productivity (Relevant Samples/J)"] * 1e3

0    0.065387
1    0.130951
2    0.122104
3    0.082623
4    0.243685
5    0.082623
6    0.165426
7    0.307839
Name: Productivity (Relevant Samples/J), dtype: float64

In [131]:
df.to_excel("experimental_table.xlsx", index=False)

In [90]:
density_scale_model(2032)

6.499570514329353